# Adversal validation
1. Isna by axis=0\
\
First import the essentials

In [59]:
import os
import sys
import keras
import tensorflow as tf
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/packages')
    !pip install git+https://github.com/keras-team/keras-tuner.git
    !pip install autokeras
    config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} )
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
else:
    path_fill = 'data/2022/filled_trian.csv'
    path_raw_train='data/2022/train.csv'
    path_A='data/2022/test_A榜.csv'
    path_B='data/2022/test_B榜.csv'

In [60]:
import pandas as pd
import useful_functions as uf
import autokeras as ak
df_train = pd.read_csv(path_raw_train).head(500)
df_train_filled = pd.read_csv(path_fill,index_col=0).head(500)
df_test_A = pd.read_csv(path_A).head(500)
df_test_B = pd.read_csv(path_B).head(500)

In [61]:
import numpy as np

2. Drop too few columns, optional drop index for train data

In [62]:
def drop_too_few_variable(dataframe: pd.DataFrame, threshold=0.75, whether_index = False):
    total_rows = len(df_test_A.index)
    mask_drop = (dataframe.count() / total_rows ) > threshold
    dataframe = dataframe.loc[:, mask_drop]
    if whether_index:
        drop_index = dataframe.isna().sum(axis=1)>=len(dataframe.columns)*threshold
        drop_index = [ind for ind in drop_index.index if drop_index[ind]==True]
        dataframe.drop(index = drop_index)
        return dataframe.drop(index = drop_index)
    else:
        return dataframe
df_train = drop_too_few_variable(df_train,0.75,True)
portfolio:list = [df_test_A,df_test_B]
for i in range(len(portfolio)):
    portfolio[i] = drop_too_few_variable(portfolio[i])

3. step 1, minus 2

In [63]:
s_col = uf.dynamic_string_col(df_train)
df_train,string_col = uf.step1_data_processing(df_train,s_col)

# Need try iqr first or fillna first
\
\
4.  Fillna, try random_forest, KNN, mode
## 3.2 数据集中缺失值从少到多进行排序
从缺失值最少的开始填，所以首先需要一个排序，即数据集中缺失值从少到多的一个顺序
# 3.3构建新特征矩阵和新标签

In [65]:
sor_by_nan = df_train.isna().sum().sort_values(ascending=True)
con = sor_by_nan>0
sor_by_nan =sor_by_nan[con].index # index

i=10    # loop over all columns with nan 
to_be_fillc:str = sor_by_nan[i]
#---------------- FILL IN ALL ZERO --------------------------------
x_tobe_0 = sor_by_nan[sor_by_nan!=to_be_fillc]

y_tobe_trained = df_train[df_train[to_be_fillc].notna()][to_be_fillc]
tobe_tested_index = df_train[df_train[to_be_fillc].isna()][to_be_fillc].index
x_tobe_tested = df_train.loc[tobe_tested_index,x_tobe_0]
x_tobe_trained = df_train.loc[y_tobe_trained.index,x_tobe_0]
x_tobe_trained.isna().sum()

MON_12_AGV_LVE_ACT_CNT                      0
MON_12_ACM_LVE_ACT_CNT                      0
MON_12_AGV_ENTR_ACT_CNT                     0
MON_12_EXT_SAM_TRSF_IN_AMT                  0
LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV      0
LAST_12_MON_MON_AVG_TRX_AMT_NAV             0
MON_12_EXT_SAM_AMT                          0
MON_12_EXT_SAM_NM_TRSF_OUT_CNT              0
MON_12_EXT_SAM_TRSF_OUT_AMT                 0
MON_12_ACM_ENTR_ACT_CNT                     0
AI_STAR_SCO                                 4
COR_KEY_PROD_HLD_NBR                        7
WTHR_OPN_ONL_ICO                            7
PUB_TO_PRV_TRX_AMT_CUR_YEAR                 9
CUR_YEAR_COUNTER_ENCASH_CNT                 9
CUR_YEAR_MON_AGV_TRX_CNT                    9
CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT           9
CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT            9
COUNTER_CUR_YEAR_CNT_AMT                    9
REG_DT                                     20
ICO_CUR_MON_ACM_TRX_TM                     41
ICO_CUR_MON_ACM_TRX_AMT           